In [1]:

import sys, os
cdir = %pwd
sys.path.append(os.path.join(cdir, '..', 'python'))


In [2]:
from hue_show import *

In [7]:
!curl https://www.meethue.com/api/nupnp

[{"id":"001788fffe0aa167","internalipaddress":"10.0.1.16"},{"id":"001788fffe273a83","internalipaddress":"10.0.1.2"}]

In [8]:
from time import sleep
from phue import Bridge
import random
from numpy import zeros, ones
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *
import requests as req
import pandas as pd
from rgb_cie import Converter

In [9]:
bridges = req.get('https://www.meethue.com/api/nupnp').json()

In [13]:
bri = []
for br in bridges:
    b = Bridge(br['internalipaddress'])
    b.connect
    bri.append(b)

In [14]:
b1 = bri


In [15]:
ln = 1
speed = 20 # m/s
distance = 7 #m between each lamp
time = 7 / (speed * 1E3 / (60*60))

def light(name, r,g,b):
    l = b.lights_by_name[name]
    rgbl(l, 1.0,0.0,0.0, brightness=254)

normal()    
for n in range(9):
    try:
        name = "P%d"%((ln+n)%(19)+1)
        l = b.lights_by_name[name]
        rgbl(l, 1.0,0.0,0.0, brightness=254)
        sleep(time)
        rgbl(l, 1.,1.,1., brightness=254)
        sleep(0.1)
    except Exception as e:
        print(e)


TypeError: normal() missing 1 required positional argument: 'b'

### Search for lights

In [16]:
import json
import time

In [17]:
b1 = b1[0]

In [18]:
IP = b1.ip

In [58]:
get = lambda i, str_:req.get('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_).json()
post = lambda i, str_, data:req.post('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_, json.dumps(data)).json()
put = lambda i, str_,  data:req.put('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_, json.dumps(data)).json()

Search for new lights

In [48]:
post(0,'lights',{})
post(1,'lights',{})

[{'success': {'/lights': 'Searching for new devices'}}]

In [49]:
get(0,'lights/new')
get(1,'lights/new')

{'lastscan': 'active'}

In [50]:
print('now there are %d lights'%(len(get(0,'lights'))))

now there are 29 lights


In [67]:
lamps = {}
for i in range(len(bri)):
    print('Bridge-%d'%(i))
    for k,v in get(i,'lights').items():
        #if 'Hue color' in v['name']:
        print(v['name'], v['state']['reachable'], v['uniqueid'])
        idl = v['uniqueid']
        if idl in lamps:
            lamps[idl][i] = v
        else:
            lamps[idl] = {i:v}
    time.sleep(2)

Bridge-0
Hue Lamp 26 False 00:17:88:01:00:dc:f6:9b-0b
Table 2 False 00:17:88:01:00:bf:3a:24-0b
Hue Lamp 15 False 00:17:88:01:00:f5:80:43-0b
Hue Lamp 23 False 00:17:88:01:00:ec:95:ff-0b
Hue Lamp 8 True 00:17:88:01:00:dc:ea:dc-0b
Hue Lamp 19 True 00:17:88:01:00:d9:cb:35-0b
Hue Lamp 11 False 00:17:88:01:00:dc:58:1b-0b
Hue Lamp 25 False 00:17:88:01:00:f5:4a:93-0b
Hue Lamp 6 True 00:17:88:01:00:db:ad:de-0b
Hue Lamp 12 False 00:17:88:01:00:f5:80:7f-0b
Hue Lamp 20 False 00:17:88:01:00:ec:91:5b-0b
Sofa False 00:17:88:01:00:ba:99:e7-0b
Hue Lamp 7 True 00:17:88:01:00:f5:8e:ff-0b
Hue lightstrip 1 False 00:17:88:01:00:cf:a5:dc-0b
Stairs False 00:17:88:01:00:bf:2f:6d-0b
Hue Lamp 24 False 00:17:88:01:00:ec:95:8a-0b
Reol False 00:17:88:01:00:b1:f1:68-0b
Hue Lamp 27 False 00:17:88:01:00:ee:c4:2e-0b
Hue Lamp 10 True 00:17:88:01:00:d9:cb:67-0b
Hue Lamp 17 True 00:17:88:01:00:f5:71:01-0b
Hue Lamp 16 True 00:17:88:01:00:f5:92:2e-0b
Table 1 False 00:17:88:01:00:b1:f8:dc-0b
Hue Lamp 3 False 00:17:88:01:00:d

In [68]:
for k,v in lamps.items():
    if len(v)>1:
        print('0:',   v[0]['state']['reachable'], v[0]['name'], 
              '\n1:', v[1]['state']['reachable'], v[1]['name'], 
              '\n')

0: False Hue Lamp 3 
1: False P17 

0: False Hue Lamp 20 
1: False P4 

0: False Hue Lamp 15 
1: False Hue color lamp 15 

0: False Hue Lamp 26 
1: False Hue color lamp 5 

0: True Hue Lamp 6 
1: True P18 

0: False Hue Lamp 21 
1: False Hue color lamp 4 

0: True Hue Lamp 8 
1: True P16 

0: True Hue Lamp 10 
1: True P3 

0: True Hue Lamp 7 
1: True P2 

0: False Hue Lamp 9 
1: False P1 

0: False Hue Lamp 25 
1: False Hue color lamp 7 

0: True Hue Lamp 17 
1: True P12 

0: False Hue lightstrip 1 
1: False Hue lightstrip 1 

0: False Hue Lamp 13 
1: False P13 

0: False Hue Lamp 12 
1: False P14 

0: False Hue Lamp 14 
1: False P5 

0: True Hue Lamp 16 
1: True P11 

0: False Hue Lamp 27 
1: False P8 

0: False Hue Lamp 18 
1: False P9 

0: True Hue Lamp 19 
1: True P10 

0: False Hue Lamp 11 
1: False P15 

0: False Hue Lamp 24 
1: False Hue color lamp 8 

0: False Hue Lamp 22 
1: False Hue color lamp 6 



In [73]:
lamps2 = {}
for k, v in lamps.items():
    if 0 in v:
        if 'P' in v[0]['name']:
                lamps2[v[0]['name']] = v
    if 1 in v:
        if 'P' in v[1]['name']:
            lamps2[v[1]['name']] = v
        

In [75]:
lamps2['P1']

{0: {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'Hue Lamp 9',
  'pointsymbol': {'1': 'none',
   '2': 'none',
   '3': 'none',
   '4': 'none',
   '5': 'none',
   '6': 'none',
   '7': 'none',
   '8': 'none'},
  'state': {'alert': 'none',
   'bri': 0,
   'colormode': 'hs',
   'ct': 0,
   'effect': 'none',
   'hue': 2000,
   'on': True,
   'reachable': False,
   'sat': 0,
   'xy': [0.0, 0.0]},
  'swversion': '66013187',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:d9:c9:cf-0b'},
 1: {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'P1',
  'state': {'alert': 'none',
   'bri': 254,
   'colormode': 'hs',
   'ct': 187,
   'effect': 'none',
   'hue': 2000,
   'on': True,
   'reachable': False,
   'sat': 213,
   'xy': [0.3362, 0.3604]},
  'swversion': '66013187',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:d9:c9:cf-0b'}}

In [25]:
all_lights = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", "5D3F46", "132A13", "81D5FE", "CF5C37"]

In [26]:
len(all_lights[0:10])

10

In [27]:
import json

In [28]:
import time

In [53]:
for i in range(len(bri)):
    post(i,'lights', {"deviceid":all_lights[0:10]})
    time.sleep(2)
    post(i,'lights', {"deviceid":all_lights[10:]})
    time.sleep(2)
    

### PARTY MODE

In [66]:
for i in range(len(bri)):
    put(i, 'groups/0/action', 
        {"on": True,
         "hue": 2000,
        })
    time.sleep(1)

In [ ]:
class Controler(object):
    def __init__(self):
        self.targets = {}
        